<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum

import pandas as pd
import matplotlib.pyplot as plt





In [2]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')


In [5]:
# Seasons worked
for df in [df_val,df_train]:
    seasons = df.groupby("directors").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["seasonsOfExperience"] = df["directors"].apply(sum_into_column,args=(seasons,))

# episodes worked
for df in [df_val,df_train]:
    episodes = df.groupby("directors").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["episodesOfExperience"] = df["directors"].apply(sum_into_column,args=(episodes,))
    
# movies worked
for df in [df_val,df_train]:
    movies = df[df["titleType"] == "movie"].groupby("directors").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["moviesOfExperience"] = df["directors"].apply(sum_into_column,args=(movies,))

# other xp than movies
for df in [df_val,df_train]:
    others = df[df["titleType"] != "movie"].groupby("directors").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["othersOfExperience"] = df["directors"].apply(sum_into_column,args=(others,))

del seasons,episodes,movies,others,df

In [6]:
def give_authors_experience(writer):
    authors = []
    if type(writer) != str:
        return 0
    
    for i,author in enumerate(writer.split(',')):
        authors.append(writer_counts[author])
    
    return sum(authors)

for df in [df_train, df_val]:

    writers = df['writers']
    writers = writers.str.split(',', expand=True)
    writer_counts = writers.stack().value_counts().to_dict()
    writer_counts['0'] = 0

    df['writers_exp'] = df['writers'].apply(give_authors_experience)


del writers, writer_counts,df

MemoryError: Unable to allocate 9.38 GiB for an array with shape (977541, 1288) and data type object

In [ ]:
def give_directors_experience(director):
    directors = []
    if type(director) != str:
        return directors
    
    for i,author in enumerate(director.split(',')):
        directors.append(directors_counts[author])
    
    return sum(directors)

for df in [df_train, df_val]:
        
    directors = df['directors']
    directors = directors.str.split(',', expand=True)
    directors_counts = directors.stack().value_counts()
    directors_counts['0'] = 0


    #apply the function to the writers column, send also the ratings column to the function
    df['directors_exp'] = df['directors'].apply(give_directors_experience)



del directors, directors_counts, df

In [ ]:
#set language to 0 if it is nan

df_train['language'] = df_train['language'].fillna(0)
df_val['language'] = df_val['language'].fillna(0)

In [ ]:
y=df_train.averageRating
vars=['startYear', 'runtimeMinutes',"numVotes","writers_exp",
      "directors_exp","endYear","isAdult","seasonNumber",
      "episodeNumber","ordering","isOriginalTitle","runtime",
      "revenue","popularity","budget","seasonsOfExperience",
      "episodesOfExperience","othersOfExperience","moviesOfExperience"]
X=df_train[vars]
X = pd.concat([X, pd.get_dummies(df_train['language'], prefix='language')], axis=1)
X_pred=df_val[vars]
X_pred = pd.concat([X_pred, pd.get_dummies(df_val['language'], prefix='language')], axis=1)


In [ ]:
for i,df in enumerate([df_train, df_val]):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)


    title_df = pd.concat([title_df, genres], axis=1)
    if not i:
        X = pd.concat([X, title_df], axis=1)
    else:
        X_pred = pd.concat([X_pred, title_df], axis=1)


for col in X.columns:
    if col not in X_pred.columns:
        X_pred[col] = 0

X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)
del title_df, genres, title, encoder, df





In [ ]:
#drop X column named language_0
X = X.drop(columns=['language_0'])
X_pred = X_pred.drop(columns=['language_0'])
print(X.columns)

In [ ]:
X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)